# Corner detection and 3D calibration
Based on the tutorial [here](https://mecaruco2.readthedocs.io/en/latest/notebooks_rst/Aruco/sandbox/ludovic/aruco_calibration_rotation.html)

This is an improvement on the previous notebook. In this notebook, the stereo images are processed as pairs.   This is important because during the corner mapping, only the corners that appear in both images must be saved.  

In [ ]:
user = 'gerrie'
#user = 'marcvanzyl'

In [ ]:
import numpy as np
import cv2, os
from cv2 import aruco
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
%matplotlib inline

# The board
The following line of the board was updated to reflect the correct scale of the board.  This is necessary becuase the `board` object is used later   
`board = aruco.CharucoBoard_create(7, 5, .04026, .8*.04026, aruco_dict)`

Notice the board object returned.  Is contains all the magic of the CharUco pattern. For the image interpretation and to calibrate the cameras the cameras need a picture of the board and also information about the picture (ie. the details of the board).  This is all contained in the board object. 

In [ ]:
board_size = 'experimental setup'

if board_size == '7x5':
    chessboard_num_squares_across = 7
    chessboard_num_squares_up = 5
    chessboard_square_size = 0.04026
    chessboard_aruco_ratio = 0.8   # this is a fraction of chessboard_square_size
    aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)


if board_size == '12x8':
    chessboard_num_squares_across = 12
    chessboard_num_squares_up = 8
    chessboard_square_size = 1
    chessboard_aruco_ratio = 0.7   # this is a fraction of chessboard_square_size
    aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_250)

elif board_size == 'experimental setup':   
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)





The board object contains all the vectors (pointing from the bottom left corner) to each of the corners present on the board.  The two kinds of objects are **aruco markers** and the **checkerboard**.  The aruco markers are called '`markers`' in the code and documentation.  The term `marker corners` means the set of 4 corners around each aruco maker. 

The aruco markers can be extracted from the board using the `aruco.getBoardObjectAndImagePoints(board, makerCorners command)`. 

Once the algorithm detected the markerCorners then it can interpolate between the marker corners to find the checkerboard corners.  The positions of checkerboard "inside" corners can be extracted using the folowing. 

The corners are labeled starting from 0 (bottom left) and going right 

### Now using Charuco

This function:
1. finds the locations of the corners of the aruco squares (`cv2.aruco.detectMarkers`)
1. if markers were found interpolates to find the checkerboard markers between them (`cv2.aruco.interpolateCornersCharuco`)
1. zooms into each checkerboard corner to get sub-pixel accuracy using (`cv2.cornerSubPix`)

In [ ]:
def find_checkerboard_corners(img, board, clipLimit=2.0, verbose=False):

    # These are parameters used by the cv2.cornerSubPix function
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)

    # increase the contraxt
    img = increase_contrast(img, clipLimit=clipLimit)
 
    # convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    detect_params = aruco.CORNER_REFINE_SUBPIX

    # find the aruco corners and the ids of each corner
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict, detect_params)
    
    if verbose:
        print('Found {} aruco marker corners'.format(len(ids)))
    
    if len(ids)>0:
        (retval, charucoCorners,
         charucoIds) = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, board, )
        if verbose:
            print('Found {} checker corners'.format(len(charucoIds)))
        if len(charucoIds)>0:
            # SUB PIXEL DETECTION
            for corner in charucoCorners:
                if verbose:
                    print('Sub pixel optimization:')
                    print(corner)
                cv2.cornerSubPix(gray, corner,
                                 winSize = (5,5),
                                 zeroZone = (-1,-1),
                                 criteria = criteria)
                if verbose:
                    print(corner)
                    print('+++')
        
    return charucoCorners, charucoIds, gray.shape

        

One big problem is that you need to give the stereo calibration only coners that appear in both cameras.  Fortunately, the detection returns the ids of the corners in the `charucoIds`. These `charucoIds` correspond to the numbering system mentioned above (bottom left is 0 and starts going across to the right)

In [ ]:
def find_common_corners(cornersL, idsL, cornersR, idsR, verbose=False):
    # find the list of corners common to both images
    # get a list of the ids
    listIdsL = idsL.reshape((1,-1))[0].tolist()
    listIdsR = idsR.reshape((1,-1))[0].tolist()
    
    # this selects only the points that are in listIdsA and listIdsC
    commonIds = list(set(listIdsL).intersection(set(listIdsR)))

    if verbose:
        print('Found {} common IDS between L:({}) and R:({})'.format(len(commonIds),
                                                                     len(idsL),
                                                                     len(idsR)))

    # Reindex L camera data by taking out only the common points 
    re_index = [ listIdsL.index(ind) for ind in commonIds]
    idsL_new = idsL[re_index]
    cornersL_new = np.array(cornersL)[re_index]
    cornersL_new = [x for x in cornersL_new]

    # Reindex R camera data by taking out only the common points
    re_index = [ listIdsR.index(ind) for ind in commonIds]
    idsR_new = idsR[re_index]
    cornersR_new = np.array(cornersR)[re_index]
    cornersR_new = [x for x in cornersR_new]

    return np.array(cornersL_new), idsL_new, np.array(cornersR_new), idsR_new

In [ ]:
import pickle

cam_calOF = pickle.load( open('OFCameraCalibration.p', 'rb'))



In [ ]:
cam_calOF.keys()

# Now we can check

## Camera features:
- sensor size = 3.68 x 2.76 mm  
- sensor resolution  = 3280 × 2464
- focal length = 3.04 mm

$$ d_{mm} = \frac{pix \times 3.68}{3280} $$

The depth can now be found
$$ Z = \frac{T \times f}{d_{mm}} $$

In [ ]:
import pandas as pd

In [ ]:
# adapted from here https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv
def increase_contrast(img, clipLimit=3.0, verbose=False):

    if clipLimit>0.0:
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        if verbose:
            cv2.imshow("lab",lab)

        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        if verbose:
            cv2.imshow('l_channel', l)
            cv2.imshow('a_channel', a)
            cv2.imshow('b_channel', b)

        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=(8,8))
        cl = clahe.apply(l)
        if verbose:
            cv2.imshow('CLAHE output', cl)

        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        if verbose:
            cv2.imshow('limg', limg)

        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        if verbose:
            cv2.imshow('final', final)
    else:
        final = img.copy()

    return final

In [ ]:
def find_charuco_marker_corners(img, aruco_dict, clipLimit=3.0, verbose=False):

    # These are parameters used by the cv2.cornerSubPix function
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10000, 1e-11)

    # increase the contraxt
    img = increase_contrast(img, clipLimit=clipLimit)


 
    # convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    

    # find the aruco corners and the ids of each corner
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict)

    if verbose:
        print('Found {} aruco marker corners'.format(len(ids)))
        
    
    #  now find the central points of the aruco markers by averaging the four corner points
    if  len(ids) > 0:
        # SUB PIXEL DETECTION
        for corner in corners:
            if verbose:
                print('Sub pixel optimization:')
                print(corner)
            cv2.cornerSubPix(gray, corner,
                             winSize = (4,4),
                             zeroZone = (-1,-1),
                             criteria = criteria)
            if verbose:
                print(corner)
                print('+++')




    return np.array(corners), ids, gray.shape

        

In [ ]:
def find_aruco_center(corners, ids):
    # find the center point of the 4 corner of the aruco markers
    centers = []
    for cnrs in corners:
        center = np.array((np.average(cnrs[0][:,0]), np.average(cnrs[0][:,1])))
        centers.append(center)
    centers = np.array(centers)
    centers = np.array(centers).reshape((-1,1,2))
    center_ids = np.arange(centers.shape[0]).reshape(-1,1)
    
    return centers, ids

In [ ]:
image_number = 0

# Load the list of files

In [ ]:
datadir = "/Users/{}/Google Drive/ScienceFair2021/Random_video/500/"
datadir = "/Users/{}/Google Drive/ScienceFair2021/DataCapture/realData/".format(user)

video_files = np.array([f for f in os.listdir(datadir) if f.endswith(".mp4") ])



In [ ]:
video_files[3]

In [ ]:
video_capture = cv2.VideoCapture('{}{}'.format(datadir, 'Rotate_detailed_0_-5-3-00908I.mp4'))
#video_capture.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'h264'))

video_capture.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'H264'))

#cap.set(CV_CAP_PROP_FOURCC, CV_FOURCC('H', '2', '6', '4'));
#video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
#video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

frame = 0
while video_capture.isOpened():
    ret, image = video_capture.read()
    
    
    if not ret:
        break
    if frame%20 == 0:
        image2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        plt.imshow(image2,cmap='gray')
        plt.title('Frame {}'.format(frame))
        plt.show()
        plt.imshow(cv2.cvtColor(increase_contrast(image, clipLimit=3.0, verbose=False), cv2.COLOR_BGR2GRAY),cmap='gray')
        plt.title('Frame {}'.format(frame))
        plt.show()
    frame += 1
print('Done!')

# Run the analysis

In [ ]:
from astropy.nddata.blocks import block_reduce

In [ ]:
# set verbose = True to make plots of each frame
verbose =  True

for file in [video_files[3]]:
    
    result_list = []

    counter = 0
    
    # the size of the square to average the flow over
    center_range = 10

    cap = cv2.VideoCapture(datadir + file)
    print("Working with: {}".format(file))
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # discard the first 20 frames becuase the movment may be accelerating
    for i in range(20):
        ret, curr_pic_color = cap.read()

    ret, frame1 = cap.read()
    last_pic_3_color = cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
    last_pic_3_grey = cv2.cvtColor(last_pic_3_color,cv2.COLOR_BGR2GRAY)


    ret, frame1 = cap.read()
    last_pic_2_color = cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
    last_pic_2_grey = cv2.cvtColor(last_pic_2_color,cv2.COLOR_BGR2GRAY)
    

    ret, frame1 = cap.read()
    last_pic_1_color = cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
    last_pic_grey = cv2.cvtColor(last_pic_1_color,cv2.COLOR_BGR2GRAY)

    last_pic_grey = last_pic_grey[:,200:-200]

    

    hsv = np.zeros_like(frame1[:,200:-200,:])
    hsv[...,1] = 255
    flow = np.zeros_like(frame1[:,200:-200,:])

    # this creates the array for the sorted centers 
    sorted_centers = np.zeros([10,2])

    while(cap.isOpened()):

        ret, curr_pic_color = cap.read()
        ret, curr_pic_color = cap.read()
        ret, curr_pic_color = cap.read()




        # ret will be False if there are no more frames
        if ret == False:
            break
            
        counter += 1
        if counter > 120:
            break
        curr_pic_color = cv2.undistort(curr_pic_color,cam_calOF['mtx'],cam_calOF['dist'],None)
        curr_pic_grey = cv2.cvtColor(curr_pic_color, cv2.COLOR_BGR2GRAY)
        curr_pic_grey = curr_pic_grey[:,200:-200]

        if counter > 1:
            flags = cv2.OPTFLOW_USE_INITIAL_FLOW
        else:
            flags = 0

        flow = cv2.calcOpticalFlowFarneback(last_pic_grey, curr_pic_grey, flow, 0.5, 3, 50, 6, 7, 1.2, flags)

        # clip flows to clean up the data
        flow[flow>100] = 0


        if verbose:
            print("Flow: mean: {} max: {}  min: {} {}".format(flow.mean(), flow.max(), flow.min(), flow.shape))


            mag = (cv2.multiply(cv2.add(flow[...,0],0.),6.))

            mag[mag<0.0] = 0
            mag[mag>255] = 255

            print("Mag: mean: {} max: {}  min: {} {}".format(mag.mean(), mag.max(), mag.min(), flow.shape))

            # edge detection for the outlines
            edges = cv2.Canny(curr_pic_grey, 50, 100)


            hsv[...,0] = 0
            #hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
            hsv[...,2] = mag
            rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

            annot_font = {'fontname':'Arial', 'size':'14','weight':'bold'}

            fig, axs = plt.subplots(figsize=(36,24))
            rgb[edges>100,1] = 255 
            axs.imshow(rgb)

            plt.show()

            flow_x = flow[:,:,0]
            flow_y = flow[:,:,1]

            flow_x_s = block_reduce(flow_x.reshape((2464, 2864)), 64, func=np.mean).T
            flow_y_s = block_reduce(flow_y.reshape((2464, 2864)), 64, func=np.mean).T


            x_ticks = flow_x_s.shape[0]
            y_ticks = flow_x_s.shape[1]


            X, Y = np.mgrid[0:x_ticks, 0:y_ticks]

            fig, axs = plt.subplots(figsize=(36,24))
            axs.quiver(X, Y, flow_x_s*4, flow_y_s*4, alpha=.5)


            plt.show() 

        last_pic_grey = np.copy(curr_pic_grey)
        
    pickle.dump(np.array(result_list)[5:105], open('{}{}_res.p'.format(datadir, file[:-5]), 'wb'))

In [ ]:
edge = int(flow.shape[1]/2-200)


In [ ]:
flow_slice = flow[:,edge:-edge,:]

In [ ]:
flow_slice.shape

In [ ]:
slice_mean = np.mean(flow_slice[:,:,0])


In [ ]:
slice_std = np.std(flow_slice[:,:,0])

In [ ]:
flow_slice_clean = flow_slice[:,:,0].copy()

In [ ]:
flow_slice_clean[flow_slice_clean>slice_mean+2*slice_std] = np.nan

In [ ]:
# set verbose = True to make plots of each frame
verbose =  True

for file in [video_files[3]]:
    
    result_list = []

    counter = 0
    #2175
    top = []
    #1087
    middle = []
    #0
    bottom = []
    # the size of the square to average the flow over
    center_range = 10

    cap = cv2.VideoCapture(datadir + file)
    print("Working with: {}".format(file))
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # discard the first 20 frames becuase the movment may be accelerating
    for i in range(20):
        ret, curr_pic_color = cap.read()

    ret, frame1 = cap.read()
    last_pic_3_color = cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
    last_pic_3_grey = cv2.cvtColor(last_pic_3_color,cv2.COLOR_BGR2GRAY)


    ret, frame1 = cap.read()
    last_pic_2_color = cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
    last_pic_2_grey = cv2.cvtColor(last_pic_2_color,cv2.COLOR_BGR2GRAY)
    

    ret, frame1 = cap.read()
    last_pic_1_color = cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
    last_pic_grey = cv2.cvtColor(last_pic_1_color,cv2.COLOR_BGR2GRAY)
    

    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    flow = np.zeros_like(frame1)

    # this creates the array for the sorted centers 
    sorted_centers = np.zeros([10,2])

    while(cap.isOpened()):

        ret, curr_pic_color = cap.read()
        ret, curr_pic_color = cap.read()
        




        # ret will be False if there are no more frames
        if ret == False:
            break
            
        counter += 1
        
        threshold = int((3/8)*length)
        mid = length/2
        if mid+threshold < counter:
            break
        elif counter == mid-threshold-1:
            ret, frame1 = cap.read()
            last_pic_1_color = cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
            last_pic_grey = cv2.cvtColor(last_pic_1_color,cv2.COLOR_BGR2GRAY)
        elif counter > mid-threshold:
            print(counter)
            ret, curr_pic_color = cap.read()
            curr_pic_color = cv2.undistort(curr_pic_color,cam_calOF['mtx'],cam_calOF['dist'],None)
            curr_pic_grey = cv2.cvtColor(curr_pic_color, cv2.COLOR_BGR2GRAY)

            if counter > 1:
                flags = cv2.OPTFLOW_USE_INITIAL_FLOW
            else:
                flags = 0

            flow = cv2.calcOpticalFlowFarneback(last_pic_grey, curr_pic_grey, flow, 0.5, 3, 50, 6, 7, 1.2, flags)

            # clip flows to clean up the data
            mean = np.mean(flow)
            print("Mean of frame: {}".format(mean))
            flow[flow>100] = 0


            if verbose:
                print("Flow: mean: {} max: {}  min: {} {}".format(flow.mean(), flow.max(), flow.min(), flow.shape))


                mag = (cv2.multiply(cv2.add(flow[...,0],0.),6.))

                mag[mag<0.0] = 0
                mag[mag>255] = 255

                print("Mag: mean: {} max: {}  min: {} {}".format(mag.mean(), mag.max(), mag.min(), flow.shape))

                # edge detection for the outlines
                edges = cv2.Canny(curr_pic_grey, 50, 100)


                hsv[...,0] = 0
                #hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                hsv[...,2] = mag
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

                annot_font = {'fontname':'Arial', 'size':'14','weight':'bold'}

                fig, axs = plt.subplots(figsize=(36,24))
                rgb[edges>100,1] = 255 
                axs.imshow(rgb)

                plt.show()

                flow_x = flow[:,:,0]
                flow_y = flow[:,:,1]

                flow_x_s = block_reduce(flow_x.reshape((2464, 3264)), 32, func=np.mean).T
                flow_y_s = block_reduce(flow_y.reshape((2464, 3264)), 32, func=np.mean).T


                x_ticks = flow_x_s.shape[0]
                y_ticks = flow_x_s.shape[1]


                X, Y = np.mgrid[0:x_ticks, 0:y_ticks]

                fig, axs = plt.subplots(figsize=(36,24))
                axs.quiver(X, Y, flow_x_s, flow_y_s, alpha=.5)


                plt.show() 

            last_pic_grey = np.copy(curr_pic_grey)

    pickle.dump(np.array(result_list)[5:105], open('{}{}_res.p'.format(datadir, file[:-5]), 'wb'))

In [ ]:
flow.shape

In [ ]:
from astropy.nddata.utils import block_reduce
flow_x = flow[:,:,0]
flow_y = flow[:,:,1]

flow_x_s = block_reduce(flow_y.reshape((2464, 3264)), 32, func=np.mean)
flow_y_s = block_reduce(flow_y.reshape((2464, 3264)), 32, func=np.mean)


x_ticks = flow_x_s.shape[0]
y_ticks = flow_x_s.shape[1]


X, Y = np.mgrid[0:x_ticks, 0:y_ticks]


plt.quiver(X, Y, flow_x_s, flow_y_s, scale=1, alpha=.5)

plt.xlim(-1, n)
plt.xticks([])
plt.ylim(-1, n)
plt.yticks([])

plt.show()


In [ ]:
Y


In [ ]:
flow_x_ss = block_reduce(flow_x_s.reshape((1232, 1632)), 2, func=np.mean)

In [ ]:
flow_x_s.shape

In [ ]:
arr1inds = center_ids.argsort()
print(arr1inds)

In [ ]:
sorted_centers = centers[arr1inds]
sorted_centers

In [ ]:
center_ids[arr1inds]

In [ ]:
pickle.dump(np.array(result_list), open('{}{}_res.p'.format(datadir, file[:-5]), 'wb'))

In [ ]:
'{}{}_res.p'.format(datadir, file[:-5])

In [ ]:
np.array(result_list)[5:105].shape

In [ ]:
rgb

In [ ]:
flow[...,0].mean()

In [ ]:
flow[...,1].mean()

In [ ]:
sorted_centers


In [ ]:
flow.shape

In [ ]:
centerx = 100
centery = 120

flow[centery-10:centery+10,centerx-10:centerx+10][:,:,0].mean()

In [ ]:
flow[centery-10:centery+10,centerx-10:centerx+10][:,:,1].mean()

In [ ]:
sorted_centers.shape

In [ ]:
np.argsort(center_idsL.reshape(-1)[2:])

In [ ]:
centersL[center_idsL.reshape(-1)[2:]]

In [ ]:
sorted_centers = np.zeros([10,2])
sorted_centers

In [ ]:
counter = 0
for i in center_idsL[1:]:
    sorted_centers[i] = centersL[1:][counter]
    counter += 1

In [ ]:
sorted_centers

In [ ]:
centersL

In [ ]:
center_idsL

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
results = pd.DataFrame(columns=['Frame', 'ArucoId', 'XPos', 'YPos', 'Pos', 'File'])

for i, points in enumerate(centersAll):
    for j, point in enumerate(points): 
        ser = pd.Series()
        ser['File'] = "images[i]"
        ser['Frame'] = i
        ser['ArucoId'] = centersIdsAll[i][j][0]
        ser['Pos'] = point[0]
        ser['XPos'] = point[0][0]
        ser['YPos'] = point[0][1]

        #results = results.append(ser,ignore_index=True)
        results.loc[i*10+j] = ser
    

In [ ]:
results.shape

In [ ]:
results_003 = results.copy()

In [ ]:
pickle.dump(results_003, open('results_003.p', 'wb'))

In [ ]:
actual_distance = np.array([1169.66362686, 1405.20212069, 1654.65192714, 1912.57758013,
       2175.9671413 , 2443.05403133, 2531.44557732, 2531.73156376,
       2136.68322387, 2135.49530535])

## Camera features:
- sensor size = 3.68 x 2.76 mm  
- sensor resolution  = 3280 × 2464
- focal length = 3.04 mm

$$ d_{mm} = \frac{pix \times 3.68}{3280} $$

The depth can now be found
$$ Z = \frac{T \times f}{d_{mm}} $$

In the case of optical flow $T$ is the distance travelled between frames

In [ ]:
camera_focal_length = 3.04
sensor_size_x = 3.68
sensor_resolution_x = 3280

In [ ]:
actual_distance[0]

In [ ]:
results.shape

In [ ]:
grp = results.groupby('ArucoId')

In [ ]:
grp.groups.keys()

In [ ]:
results = results[results['ArucoId']<10]

In [ ]:
results

In [ ]:
col_index = pd.MultiIndex.from_tuples([('15mm/s', 'Distance'),('15mm/s', 'Error'),('15mm/s', 'Std Dev')], names=['Camera Speed', 'Metric'])
final =  pd.DataFrame(columns = col_index)


for step in range(1,6):
    # create the heading for the results dataframe
    speed_str = '{}mm/s'.format(step*15)
    # this is the distance the camera traveled each frame
    camera_distance = step
    
    step_results = results[results['Frame']%step == 0].copy()
    step_grp = step_results.groupby('ArucoId')
    for marker_id in step_grp.groups.keys():
        aruco_id = step_grp.get_group(marker_id)
        aruco_id['DisparityPixel'] = aruco_id['XPos'].diff()
        aruco_id['CameraSteps'] = aruco_id['Frame'].diff()/step    
        print(aruco_id.shape)

        aruco_id['DisparityPixelPerStep'] = aruco_id['DisparityPixel']
        aruco_id['DisparityMM'] = aruco_id['DisparityPixelPerStep']*sensor_size_x/sensor_resolution_x
        distance = camera_distance*camera_focal_length/aruco_id['DisparityMM'].mean()
        std_dev = camera_focal_length/aruco_id['DisparityMM'].std()/camera_distance
        error = (camera_distance*camera_focal_length/aruco_id['DisparityMM']) - actual_distance[marker_id]
        mean_error = error[abs(error)<1000 ].mean(skipna=True)
        std_dev_error = error[abs(error)<1000 ].std()

        final.loc[marker_id, (speed_str, 'Distance')] = round(distance,1)
        final.loc[marker_id, (speed_str, 'Error')] = round(mean_error,1)
        final.loc[marker_id, (speed_str, 'Std Dev')] = round(std_dev_error, 1)
        final.loc[marker_id, (speed_str, 'Max')] =  round(error[abs(error)<1000 ].max(), 1)
        final.loc[marker_id, (speed_str, 'Min')] = round(error[abs(error)<1000 ].min(),1)



In [ ]:
final

In [ ]:
aruco_id['DisparityPixelPerStep'].plot()

In [ ]:
error.plot()

In [ ]:
final.index.name = 'Target'

In [ ]:
final

In [ ]:
final_003 = final.copy()

In [ ]:
import pickle

In [ ]:
pickle.dump(final_003, open('of_final_003.p', 'wb'))